<a href="https://colab.research.google.com/github/busranur-sr/ClimateChange/blob/main/co2regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [190]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.decomposition import PCA,TruncatedSVD

%matplotlib inline


In [191]:
df = pd.read_csv('out.csv')

df['Date']=df['Date'].str.split(expand=True)[0] # Date verisi object halindeydi. İçinde veri "1990 [YR1990]" bu şekilde bulunuyordu. İlk kısım çekildi.	
df=df.replace('..', np.nan) # Boş veriler .. şeklinde gösteriliyordu. Nan haline getirildi.
df=df.rename(columns={"CO2 emissions from liquid fuel consumption (kt)": "Liquid Co",  # Column isimlerinin düzenlenmesi
                      "CO2 emissions from gaseous fuel consumption (kt)" :"Gaseous Co",
                      "CO2 emissions from solid fuel consumption (kt)" : "Solid Co",
                      "CO2 emissions (kt)":"Total Co"})
s = df.drop('Country Name',1).select_dtypes(include='object').columns  # data type object-> float. Üzerinde işlem yapabilmek için 
df[s] = df[s].astype("float")
df=df.astype({'Date':'int16','Country Name':'str'})
df=df[df['Date']<2016] # Rest of years's data is empty


In [192]:
nan_count = df.isnull().sum()
drop_column = nan_count[nan_count<600]
display(drop_column)
df = df[drop_column.index]
df = df.dropna()
display(df)

Country Name                                                                                   0
Date                                                                                           0
Access to electricity (% of population)                                                      196
Agricultural land (% of land area)                                                            97
Agricultural land (sq. km)                                                                    91
Agriculture, forestry, and fishing, value added (% of GDP)                                   256
Arable land (% of land area)                                                                 116
CO2 emissions (kg per 2015 US$ of GDP)                                                       264
CO2 emissions (kg per 2017 PPP $ of GDP)                                                     294
CO2 emissions (kg per PPP $ of GDP)                                                          277
Total Co                      

,Country Name,Date,Access to electricity (% of population),Agricultural land (% of land area),Agricultural land (sq. km),"Agriculture, forestry, and fishing, value added (% of GDP)",Arable land (% of land area),CO2 emissions (kg per 2015 US$ of GDP),CO2 emissions (kg per 2017 PPP $ of GDP),CO2 emissions (kg per PPP $ of GDP),Total Co,CO2 emissions (metric tons per capita),CO2 emissions from gaseous fuel consumption (% of total),Gaseous Co,CO2 emissions from liquid fuel consumption (% of total),Liquid Co,CO2 emissions from solid fuel consumption (% of total),Solid Co,Cereal yield (kg per hectare),Electricity production from coal sources (% of total),Electricity production from hydroelectric sources (% of total),Electricity production from natural gas sources (% of total),Electricity production from oil sources (% of total),"Electricity production from renewable sources, excluding hydroelectric (% of total)","Electricity production from renewable sources, excluding hydroelectric (kWh)","Foreign direct investment, net inflows (% of GDP)",Forest area (% of land area),Forest area (sq. km),Methane emissions (kt of CO2 equivalent),"Mortality rate, under-5 (per 1,000 live births)",Nitrous oxide emissions (thousand metric tons of CO2 equivalent),"Other greenhouse gas emissions, HFC, PFC and SF6 (thousand metric tons of CO2 equivalent)",Population growth (annual %),"Population, total",Renewable electricity output (% of total electricity output),Renewable energy consumption (% of total final energy consumption),Total greenhouse gas emissions (kt of CO2 equivalent),Urban population,Urban population (% of total population),Urban population growth (annual %)
37,Albania,2000,100.000000,41.751825,11440.0,24.515361,21.094891,0.523296,0.174137,0.265687,3170.0,1.026213,0.694069,22.002,89.766309,2845.592,2.197886,69.673,3175.1,0.000000,96.149016,0.0,3.850984,0.000000,0.0,4.108776,28.076642,7693.000,3750.0,27.2,1180.0,150.656036,-0.637357,3089027.0,96.149016,41.445000,8220.0,1289391.0,41.741,0.742479
38,Albania,2011,100.000000,43.832117,12010.0,18.226765,22.700730,0.471671,0.156958,0.169952,5040.0,1.734823,0.582063,29.336,74.940675,3777.010,10.622659,535.382,4750.7,0.000000,98.592221,0.0,1.407779,0.000000,0.0,8.135333,28.594653,7834.935,3120.0,12.1,1170.0,1014.183540,-0.269017,2905195.0,98.592221,35.961102,9630.0,1546929.0,53.247,1.787784
39,Albania,2012,99.900000,43.843066,12013.0,18.766799,22.594891,0.422631,0.140639,0.150014,4580.0,1.579092,0.640524,29.336,68.936397,3157.287,13.691201,627.057,4883.8,0.000000,100.000000,0.0,0.000000,0.000000,0.0,7.451357,28.646606,7849.170,3170.0,11.2,1180.0,681.643684,-0.165151,2900401.0,100.000000,40.046700,9240.0,1575788.0,54.330,1.848379
40,Albania,2013,100.000000,43.332117,11873.0,19.565176,22.521898,0.437624,0.145628,0.156516,4790.0,1.654524,0.688998,33.003,69.588789,3333.303,11.483299,550.050,4949.8,0.000000,100.000000,0.0,0.000000,0.000000,0.0,9.816290,28.698558,7863.405,3190.0,10.4,1150.0,715.218840,-0.183211,2895092.0,100.000000,41.286999,9440.0,1603505.0,55.387,1.743639
41,Albania,2014,99.950000,42.857299,11742.9,19.990154,22.467153,0.468595,0.155934,0.160471,5220.0,1.806789,1.123985,58.672,73.972241,3861.351,13.417567,700.397,4892.6,0.000000,100.000000,0.0,0.000000,0.000000,0.0,8.693041,28.750511,7877.640,3230.0,9.9,1170.0,709.886387,-0.207047,2889104.0,100.000000,38.697399,9930.0,1630119.0,56.423,1.646116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3182,Zimbabwe,2011,36.900000,42.393693,164000.0,8.665865,10.856921,0.713597,0.348573,0.423960,11490.0,0.891090,0.000000,0.000,28.404091,3263.630,50.999704,5859.866,587.7,41.655816,56.434462,0.0,0.488281,1.421441,131000000.0,2.441511,46.165723,178592.100,11820.0,81.4,6860.0,1469.966370,1.536406,12894323.0,57.855903,79.509903,30550.0,4257061.0,33.015,0.989668
3183,Zimbabwe,2012,44.000000,41.876696,162000.0,8.044518,10.339925,0.644667,0.314

In [193]:
#Columnladan Total Co'nun bileşenlerini çıkardık. Aşağıdaki değerler doğrudan CP2'yi ifade ediyor.
df = df.drop(['Total greenhouse gas emissions (kt of CO2 equivalent)','Solid Co','Liquid Co','Gaseous Co','CO2 emissions (kg per 2015 US$ of GDP)',
              'CO2 emissions (kg per 2017 PPP $ of GDP)','CO2 emissions (kg per PPP $ of GDP)','CO2 emissions (metric tons per capita)'],axis = 1)
df

,Country Name,Date,Access to electricity (% of population),Agricultural land (% of land area),Agricultural land (sq. km),"Agriculture, forestry, and fishing, value added (% of GDP)",Arable land (% of land area),Total Co,CO2 emissions from gaseous fuel consumption (% of total),CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from solid fuel consumption (% of total),Cereal yield (kg per hectare),Electricity production from coal sources (% of total),Electricity production from hydroelectric sources (% of total),Electricity production from natural gas sources (% of total),Electricity production from oil sources (% of total),"Electricity production from renewable sources, excluding hydroelectric (% of total)","Electricity production from renewable sources, excluding hydroelectric (kWh)","Foreign direct investment, net inflows (% of GDP)",Forest area (% of land area),Forest area (sq. km),Methane emissions (kt of CO2 equivalent),"Mortality rate, under-5 (per 1,000 live births)",Nitrous oxide emissions (thousand metric tons of CO2 equivalent),"Other greenhouse gas emissions, HFC, PFC and SF6 (thousand metric tons of CO2 equivalent)",Population growth (annual %),"Population, total",Renewable electricity output (% of total electricity output),Renewable energy consumption (% of total final energy consumption),Urban population,Urban population (% of total population),Urban population growth (annual %)
37,Albania,2000,100.000000,41.751825,11440.0,24.515361,21.094891,3170.0,0.694069,89.766309,2.197886,3175.1,0.000000,96.149016,0.0,3.850984,0.000000,0.0,4.108776,28.076642,7693.000,3750.0,27.2,1180.0,150.656036,-0.637357,3089027.0,96.149016,41.445000,1289391.0,41.741,0.742479
38,Albania,2011,100.000000,43.832117,12010.0,18.226765,22.700730,5040.0,0.582063,74.940675,10.622659,4750.7,0.000000,98.592221,0.0,1.407779,0.000000,0.0,8.135333,28.594653,7834.935,3120.0,12.1,1170.0,1014.183540,-0.269017,2905195.0,98.592221,35.961102,1546929.0,53.247,1.787784
39,Albania,2012,99.900000,43.843066,12013.0,18.766799,22.594891,4580.0,0.640524,68.936397,13.691201,4883.8,0.000000,100.000000,0.0,0.000000,0.000000,0.0,7.451357,28.646606,7849.170,3170.0,11.2,1180.0,681.643684,-0.165151,2900401.0,100.000000,40.046700,1575788.0,54.330,1.848379
40,Albania,2013,100.000000,43.332117,11873.0,19.565176,22.521898,4790.0,0.688998,69.588789,11.483299,4949.8,0.000000,100.000000,0.0,0.000000,0.000000,0.0,9.816290,28.698558,7863.405,3190.0,10.4,1150.0,715.218840,-0.183211,2895092.0,100.000000,41.286999,1603505.0,55.387,1.743639
41,Albania,2014,99.950000,42.857299,11742.9,19.990154,22.467153,5220.0,1.123985,73.972241,13.417567,4892.6,0.000000,100.000000,0.0,0.000000,0.000000,0.0,8.693041,28.750511,7877.640,3230.0,9.9,1170.0,709.886387,-0.207047,2889104.0,100.000000,38.697399,1630119.0,56.423,1.646116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3182,Zimbabwe,2011,36.900000,42.393693,164000.0,8.665865,10.856921,11490.0,0.000000,28.404091,50.999704,587.7,41.655816,56.434462,0.0,0.488281,1.421441,131000000.0,2.441511,46.165723,178592.100,11820.0,81.4,6860.0,1469.966370,1.536406,12894323.0,57.855903,79.509903,4257061.0,33.015,0.989668
3183,Zimbabwe,2012,44.000000,41.876696,162000.0,8.044518,10.339925,12110.0,0.000000,30.523006,28.403353,696.4,39.052483,58.567858,0.0,0.554167,1.825492,168000000.0,2.044131,46.046633,178131.400,11730.0,73.8,6510.0,1827.732117,1.698084,13115149.0,60.393350,77.822197,4306228.0,32.834,1.148333
3184,Zimbabwe,2013,38.336353,41.876696,162000.0,7.144479,10.339925,12360.0,0.000000,33.287816,57.170785,669.3,45.464995,52.204807,0.0,0.595611,1.734587,166000000.0,1.954060,45.927543,177670.700,11700.0,67.8,6310.0,1972.408020,1.777672,13350378.0,53.939394,79.305901,4359432.0,32.654,1.227943
3185,Zimbabwe,2014,32.300000,41.876696,162000.0,8.745304,10.339925,12150.0,0.000000,30.694148,63.682058,824.4,43.855974,54.169160,0.0,0.538600,1.436266,144000000.0,2.4251

In [194]:
X = df.drop('Total Co', axis = 1)
X = X.drop('Country Name', axis = 1)

Y = df[['Total Co']]

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=4)
display(X_train)
display
# create a Linear Regression model object
regression_model = LinearRegression()

# pass through the X_train & y_train data set
regression_model.fit(X_train, y_train)


,Date,Access to electricity (% of population),Agricultural land (% of land area),Agricultural land (sq. km),"Agriculture, forestry, and fishing, value added (% of GDP)",Arable land (% of land area),CO2 emissions from gaseous fuel consumption (% of total),CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from solid fuel consumption (% of total),Cereal yield (kg per hectare),Electricity production from coal sources (% of total),Electricity production from hydroelectric sources (% of total),Electricity production from natural gas sources (% of total),Electricity production from oil sources (% of total),"Electricity production from renewable sources, excluding hydroelectric (% of total)","Electricity production from renewable sources, excluding hydroelectric (kWh)","Foreign direct investment, net inflows (% of GDP)",Forest area (% of land area),Forest area (sq. km),Methane emissions (kt of CO2 equivalent),"Mortality rate, under-5 (per 1,000 live births)",Nitrous oxide emissions (thousand metric tons of CO2 equivalent),"Other greenhouse gas emissions, HFC, PFC and SF6 (thousand metric tons of CO2 equivalent)",Population growth (annual %),"Population, total",Renewable electricity output (% of total electricity output),Renewable energy consumption (% of total final energy consumption),Urban population,Urban population (% of total population),Urban population growth (annual %)
2426,2011,100.000000,13.158436,2154940.000,3.150726,7.428098,54.576906,23.900884,24.100808,2260.300000,15.542150,15.749653,49.306886,2.596864,0.053371,5.620000e+08,2.692357,49.771084,8.150946e+06,814670.0,10.100000,54490.0,-121891.406250,0.077982,1.429609e+08,15.803024,3.227100,105407937.0,73.732000,0.139033
459,2012,100.000000,6.443636,577710.000,1.811346,4.186451,34.810161,44.224634,13.804364,3458.700000,10.116060,59.723302,10.220356,1.088194,3.507259,2.232900e+10,2.700169,38.730258,3.472396e+06,92550.0,5.500000,43250.0,-132315.398438,1.085817,3.471422e+07,63.230561,22.169201,28166078.0,81.137000,1.136362
2955,2012,99.893906,72.006469,338380.000,8.501467,4.128274,68.030317,28.041765,0.000000,1272.400000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000e+00,8.900000,8.782159,4.127000e+04,38830.0,41.700000,3400.0,-2442.016602,1.797217,5.267906e+06,0.000000,0.055200,2590493.0,49.175000,2.511499
769,2000,92.529496,48.494273,7677269.000,14.821280,11.634559,4.921057,25.711932,62.161307,4216.152873,67.196153,15.860375,8.905110,5.538521,1.187305,2.018700e+10,2.780148,28.734865,4.549100e+06,1471450.0,41.636774,538460.0,-279612.879607,0.992080,1.793498e+09,17.343717,32.102645,650520936.0,36.271064,3.635700
906,2015,99.255334,29.214284,8017375.626,2.158419,12.199805,34.893716,31.381193,33.632715,3789.561101,23.209582,16.184469,24.325782,1.449300,11.784492,6.250260e+11,5.251638,38.424518,1.054498e+07,1715500.0,9.700000,425420.0,-184062.903191,0.476863,9.067074e+08,28.020176,13.030476,648325155.0,71.644534,0.732237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029,2014,100.000000,71.227214,172320.000,0.754962,25.765717,33.091835,38.901049,27.602606,7696.500000,30.311698,1.757567,30.091592,0.565476,17.505286,5.869400e+10,1.921058,12.961600,3.135800e+04,53780.0,4.500000,28340.0,21276.232422,0.736464,6.460230e+07,19.262854,7.360400,53209683.0,82.365000,1.056285
2317,2000,72.496495,18.065625,231240.000,8.069375,3.445312,4.914270,81.543349,6.939311,3084.100000,1.742493,81.189113,3.957015,12.312946,0.798433,1.590000e+08,1.564790,58.826430,7.529783e+05,25580.0,38.500000,8090.0,-70.551025,1.452868,2.645994e+07,81.987546,38.572300,19326872.0,73.042000,2.017148
1441,2000,100.000000,14.425240,52580.000,1.517880,12.274348,12.706398,55.899780,31.046202,6256.600000,21.484213,7.833713,23.310713,13.227558,1.282336,1.395300e+10,0.215125,68.246914,2.487600e+05,30120.0,4.500000,26090.0,-42187.500000,0.167276,1.268430e+08,9.116049,3.776100,99760751.0,78.649000,0.327610
628,2013,61.327358,6

LinearRegression()

In [195]:
y_prediction =  regression_model.predict(X_test)
# predicting the accuracy score
score = r2_score(y_test,y_prediction)
score  

0.9861010309198169

In [204]:
svd = TruncatedSVD(n_components=5)
x = svd.fit(X).transform(X)

In [208]:
X_train, X_test, y_train, y_test = train_test_split(x, Y, test_size=0.20, random_state=1)
# create a Linear Regression model object
reg_reduction = LinearRegression()

# pass through the X_train & y_train data set
reg_reduction .fit(X_train, y_train)

LinearRegression()

In [209]:
y_prediction =  reg_reduction.predict(X_test)
svd_score = r2_score(y_test,y_prediction)
svd_score  

0.9528941275552398